In [1]:
import os, itertools, pickle
import numpy as np
import pandas as pd

from pyecharts.globals import RenderType, ThemeType
from pyecharts.charts import Map
from pyecharts import options as opts
from pyecharts.render import make_snapshot

from pyecharts.datasets import register_url

#from snapshot_phantomjs import snapshot

#from pyecharts.globals import CurrentConfig

#from snapshot_pyppeteer import snapshot
from snapshot_selenium import snapshot

# https://zhuanlan.zhihu.com/p/370741946
# https://github.com/pyecharts/pyecharts-snapshot
# https://pyecharts.org/#/zh-cn/intro
# https://pyecharts.org/#/zh-cn/render_images

from sklearn import impute, preprocessing

register_url("https://echarts-maps.github.io/echarts-china-misc-js/")

RAWDATA_PATH = "../rawData/"

In [2]:
# Get mean of air pollution
ap_d = {
    "PM25":['PM25_year_2015','PM25_year_2016','PM25_year_2017','PM25_year_2018'],
    "O3":['O3_year_2015','O3_year_2016','O3_year_2017','O3_year_2018'],
    "NO2":['NO2_year_2015','NO2_year_2016','NO2_year_2017','NO2_year_2018'],
    "NDVI_250":['NDVI_2015_250','NDV_2016_250','NDVI_2017_250','NDVI_2018_250'],
    "NDVI_500":['NDVI_2015_500','NDVI_2016_500','NDVI_2017_500','NDVI_2018_500'],
    "NDVI_1000":['NDVI_2015_1000','NDVI_2016_1000','NDVI_2017_1000','NDVI_2018_1000'],
    "PM1_wj":['wj_PM1_2014','wj_PM1_2015','wj_PM1_2016','wj_PM1_2017','wj_PM1_2018'],
    "PM10_wj":['wj_PM10_2014','wj_PM10_2015','wj_PM10_2016','wj_PM10_2017','wj_PM10_2018'],
    "NO2_wj":['wj_NO2_2014','wj_NO2_2015','wj_NO2_2016','wj_NO2_2017','wj_NO2_2018'],
    "t2m":['t2m_mean2013','t2m_mean2014','t2m_mean2015','t2m_mean2016','t2m_mean2017','t2m_mean2018']
}

province_english_map = {'重庆市':"Chongqing", '四川省':"Sichuan", 
                        '贵州省':"Guizhou", '云南省':"Yunnan", 
                        '西藏自治区':"Xizang"}

with open(os.path.join(RAWDATA_PATH, "analysis_data_no_scale.pickle"), "rb") as f:
    raw_data = pickle.load(f)

In [3]:
ap_d.keys()

dict_keys(['PM25', 'O3', 'NO2', 'NDVI_250', 'NDVI_500', 'NDVI_1000', 'PM1_wj', 'PM10_wj', 'NO2_wj', 't2m'])

In [7]:
ap_name = 'PM25'
data_pairs = raw_data.groupby("province_chinese")[ap_name].mean().reset_index().values
    
# 实例化一个Map对象
chinaMap = Map(init_opts=opts.InitOpts(width='800px', # 图表画布宽度，css 长度单位 
                                       height='600px', # 图表画布高度，css 长度单位
                                       #renderer=RenderType.CANVAS,
                                       #renderer=RenderType.SVG,
                                       #js_host="/home/data/jingbozhou/local/echarts.min.js",
                                       theme="white", # https://blog.csdn.net/qq_41595507/article/details/102947622
                                 )
              )
chinaMap.add(series_name=ap_name, 
             data_pair=data_pairs, 
             maptype="china", 
             is_map_symbol_show=False)

# 设置系列配置项
chinaMap.set_series_opts(label_opts=opts.LabelOpts(is_show=False))   # 不显示label

# 设置全局配置项
chinaMap.set_global_opts(
    title_opts=opts.TitleOpts(title=None,
                              # title 组件离容器左侧的距离。
                              # left 的值可以是像 20 这样的具体像素值，可以是像 '20%' 这样相对于容器高宽的百分比，
                              # 也可以是 'left', 'center', 'right'。
                              # 如果 left 的值为'left', 'center', 'right'，组件会根据相应的位置自动对齐。
                              pos_left='40%',
                              # title 组件离容器上侧的距离。
                              # top 的值可以是像 20 这样的具体像素值，可以是像 '20%' 这样相对于容器高宽的百分比，
                              # 也可以是 'top', 'middle', 'bottom'。
                              # 如果 top 的值为'top', 'middle', 'bottom'，组件会根据相应的位置自动对齐。
                              pos_top='10', 
                              title_textstyle_opts=opts.TextStyleOpts(font_size=14, # 文字的字体大小
                                                                      # 文字的字体系列
                                                                      # 还可以是 'serif' , 'monospace', 'Arial', 'Courier New', 
                                                                      # 'Microsoft YaHei', ...
                                                                      font_family="Arial"
                                                                     )),
    legend_opts=opts.LegendOpts(is_show=True),
    visualmap_opts=opts.VisualMapOpts(
        max_=raw_data[ap_name].max(), 
        min_=raw_data[ap_name].min(),
        #range_text=["High", "Low"],
        is_calculable=False,
        #range_color=["lightskyblue", "yellow", "orangered"],
        pos_top="middle",
        pos_left="right",
        is_piecewise=True,
        pieces=[{"value": x[1], "label":"{} ({:.2f})".format(province_english_map[x[0]], x[1])} for x in data_pairs]
    ), toolbox_opts=opts.ToolboxOpts(is_show=True))
    
    #chinaMap.render("./resPlot/{}_ChinaMap.html".format(column_name))
    
    #make_snapshot(snapshot, 
                  #chinaMap.render("./resPlot/{}_ChinaMap.html".format(column_name)), 
                  #"./{}_ChinaMap.svg".format(column_name), browser="Safari")
            
chinaMap.render_notebook()

    #chinaMap.render_notebook()
#chinaMap.render("{}.html".format(column_name))